In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_pickle('data.pandas')

In [3]:
data.head()

,address,body,date_sent,readable_date,service_center,read,type,id,ClusterId,DistrictCity,...,DATE,COUNT,PSU_ID,URB_RUR,REGION,DISTRICT,LONG,LAT,dateCollectionStartDays,_merge
0,+992935710546,Power Failure is running id 1301 | Monitoring...,2016-10-19 10:11:54,2016-10-19 10:11:54,+992939999999,0,1,1301.0,20.0,Hisor,...,1/7/2017 10:29,387.0,20.0,Rural,DRS,Gissar,68.529394,38.558734,2016-11-08,both
1,+992935710546,Power Failure is running id 1301 | Monitoring...,2016-10-19 11:11:57,2016-10-19 11:11:57,+992939999999,0,1,1301.0,20.0,Hisor,...,1/7/2017 10:29,387.0,20.0,Rural,DRS,Gissar,68.529394,38.558734,2016-11-08,both
2,+992935710546,Power Failure is running id 1301 | Monitoring...,2016-10-19 12:11:57,2016-10-19 12:11:57,+992939999999,0,1,1301.0,20.0,Hisor,...,1/7/2017 10:29,387.0,20.0,Rural,DRS,Gissar,68.529394,38.558734,2016-11-08,both
3,+992935710546,Power Failure is running id 1301 | Monitoring...,2016-10-19 13:11:58,2016-10-19 13:11:58,+992939999999,0,1,1301.0,20.0,Hisor,...,1/7/2017 10:29,387.0,20.0,Rural,DRS,Gissar,68.529394,38.558734,2016-11-08,both
4,+992935710546,Power Failure is running id 1301 | Monitoring...,2016-10-19 14:12:28,2016-10-19 14:12:28,+992939999999,0,1,1301.0,20.0,Hisor,...,1/7/2017 10:29,387.0,20.0,Rural,DRS,Gissar,68.529394,38.558734,2016-11-08,both


In [4]:
data['dateSentDays'] = data['date_sent'].dt.date
data['dateCollectionStartDays'] = pd.to_datetime(data['DateCollectionStart'], infer_datetime_format=True).dt.date
data = data[data['dateSentDays'] >= data['dateCollectionStartDays']]

In [5]:
bodyNoId = pd.Series(data['body'].str.strip().str.replace('\d+', '').str.lower().str.replace('id', '').str.replace('  ', ' '))
bodyNoId = bodyNoId.str.replace('.', '').str.replace('  ', ' ').str.replace('_ ', '').str.replace('l ', '')
bodyNoId.value_counts()

power failure is running | monitoring power ok                   41884
power failure detected                                           38433
power back detected                                              37033
power failure is running | monitoring power failure              11585
power back                                                         226
power failure                                                      224
power back detected                                                135
test sms from power failure monitor:power failure detected          54
test sms from power failure monitor:power back detected             38
test sms from power failure monitor:power failure is running         8
test sms from power failure monitor:power back detec                 2
Name: body, dtype: int64

In [6]:
power = data[['id', 'date_sent']].copy()
power['body'] = bodyNoId.copy()
power['powerBack'] = ( bodyNoId.str.contains('back') | \
                      bodyNoId.str.contains('ok') ) & \
                      ~(bodyNoId.str.contains('test'))
power['powerOff'] = ( (bodyNoId.str.contains('failure detected')) | \
    (bodyNoId.str.match('power failure')) ) & \
    ~(bodyNoId.str.contains('test'))

In [7]:
power[power.powerBack | power.powerOff].head()

,id,date_sent,body,powerBack,powerOff
54,1301.0,2016-11-08 17:52:45,power back detected,True,False
55,1301.0,2016-11-08 21:12:36,power failure detected,False,True
56,1301.0,2016-11-08 21:19:23,power failure detected,False,True
57,1301.0,2016-11-08 21:19:26,power back detected,True,False
58,1301.0,2016-11-09 17:29:17,power failure detected,False,True


In [8]:
data[(data['id'] == 1301) & \
     ((data['date_sent'].dt.date == pd.to_datetime('2016-11-14').date()) | \
      (data['date_sent'].dt.date == pd.to_datetime('2016-11-15').date()))]

,address,body,date_sent,readable_date,service_center,read,type,id,ClusterId,DistrictCity,...,COUNT,PSU_ID,URB_RUR,REGION,DISTRICT,LONG,LAT,dateCollectionStartDays,_merge,dateSentDays
81,+992935710546,Power Back detected. id 1301,2016-11-14 05:03:19,2016-11-14 05:03:19,+992939999999,0,1,1301.0,20.0,Hisor,...,387.0,20.0,Rural,DRS,Gissar,68.529394,38.558734,2016-11-08,both,2016-11-14
82,+992935710546,Power Failure detected. id 1301,2016-11-14 08:05:04,2016-11-14 08:05:04,+992939999999,0,1,1301.0,20.0,Hisor,...,387.0,20.0,Rural,DRS,Gissar,68.529394,38.558734,2016-11-08,both,2016-11-14
83,+992935710546,Power Back detected. id 1301,2016-11-14 17:59:32,2016-11-14 17:59:32,+992939999999,0,1,1301.0,20.0,Hisor,...,387.0,20.0,Rural,DRS,Gissar,68.529394,38.558734,2016-11-08,both,2016-11-14
84,+992935710546,Power Failure detected. id 1301,2016-11-14 20:39:27,2016-11-14 20:39:27,+992939999999,0,1,1301.0,20.0,Hisor,...,387.0,20.0,Rural,DRS,Gissar,68.529394,38.558734,2016-11-08,both,2016-11-14
85,+992935710546,Power Back detected. id 1301,2016-11-14 20:39:59,2016-11-14 20:39:59,+992939999999,0,1,1301.0,20.0,Hisor,...,387.0,20.0,Rural,DRS,Gissar,68.529394,38.558734,2016-11-08,both,2016-11-14
86,+992935710546,Power Failure is running id 1301 | Monitoring...,2016-11-14 21:07:46,2016-11-14 21:07:46,+992939999999,0,1,1301.0,20.0,Hisor,...,387.0,20.0,Rural,DRS,Gissar,68.529394,38.558734,2016-11-08,both,2016-11-14
87,+992935710546,Power Failure detected. id 1301,2016-11-14 22:03:23,2016-11-14 22:03:23,+992939999999,0,1,1301.0,20.0,Hisor,...,387.0,20.0,Rural,DRS,Gissar,68.529394,38.558734,2016-11-08,both,2016-11-14
88,+992935710546,Power Failure detected. id 1301,2016-11-15 08:04:08,2016-11-15 08:04:08,+992939999999,0,1,1301.0,20.0,Hisor,...,387.0,20.0,Rural,DRS,Gissar,68.529394,38.558734,2016-11-08,both,2016-11-15
89,+992935710546,Power Failure is running id 1301 | Monitoring...,2016-11-15 09:07:27,2016-11-15 09:07:27,+992939999999,0,1,1301.0,20.0,Hisor,...,387.0,20.0,Rural,DRS,Gissar,68.529394,38.558734,2016-11-08,both,2016-11-15
90,+992935710546,Power Back detected. id 1301,2016-11-15 17:57:43,2016-11-15 17:57:43,+992939999999,0,1,1301.0,20.0,Hisor,...,387.0,20.0,Rural,DRS,Gissar,68.529394,38.558734,2016-11-08,both,2016-11-15


In [131]:
"""
In case of more than one off & back per hour, we keep the last one 
  Extract date and hour and make that index, then keep max of date_sent
(since by definition, all other changes inside the hour are only minutes long)
Keep only one per hour.
To impute missing, we need to now if hours are on or off So we can only work with these messages.
Fill index again to complete the hours and merge these indicators
Start by assuming status on, then go through hours and turn indicator off if power off and on if on.
"""

'\nIn case of more than one off & back per hour, we keep the last one \n  Extract date and hour and make that index, then keep max of date_sent\n(since by definition, all other changes inside the hour are only minutes long)\nKeep only one per hour.\nTo impute missing, we need to now if hours are on or off So we can only work with these messages.\nFill index again to complete the hours and merge these indicators\nStart by assuming status on, then go through hours and turn indicator off if power off and on if on.\n'

In [9]:
power = power[power.powerBack | power.powerOff]
power['date_sent_hour'] = power['date_sent']
power.set_index('date_sent_hour', drop=True, inplace=True)
power.index = power.index.floor('H') # But doing this can create two records in the same hour
power.reset_index(inplace=True)
powerNoDup = pd.DataFrame(power[['id', 'date_sent_hour', 'date_sent']].groupby(['id', 'date_sent_hour']).max(), copy=True)
powerNoDup.reset_index(inplace=True)

In [10]:
power = power.drop(['date_sent_hour', 'body'], 1).merge(powerNoDup.drop('date_sent_hour', 1), 
                                           left_on=['id', 'date_sent'], right_on=['id', 'date_sent'], how='right')

In [11]:
power.set_index('date_sent', drop=True, inplace=True)
power.index = power.index.floor('H') 
power.reset_index(inplace=True)
power.head()

,date_sent,id,powerBack,powerOff
0,2016-11-08 17:00:00,1301.0,True,False
1,2016-11-08 21:00:00,1301.0,True,False
2,2016-11-09 17:00:00,1301.0,True,False
3,2016-11-09 21:00:00,1301.0,True,True
4,2016-11-09 22:00:00,1301.0,False,True


In [12]:
power = power.drop('powerOff', 1)
power.to_pickle('power.pandas')

In [13]:
missingHours = pd.read_pickle('missingHours.pandas')

In [14]:
merged = power.merge(missingHours, left_on=['id', 'date_sent'], right_on=['id', 'date_sent'], how='right', indicator=True)

In [15]:
merged._merge.value_counts()

right_only    611477
both           91901
left_only          0
Name: _merge, dtype: int64

In [16]:
"""
Both are hours where there was a power on or power off message
Right only, are the ones we must fill
"""

'\nBoth are hours where there was a power on or power off message\nRight only, are the ones we must fill\n'

In [17]:
# Must sort in order to fill
merged.drop('_merge', axis=1, inplace=True)
merged.set_index(['id', 'date_sent'], inplace=True)
merged.sort_index(inplace=True)

In [18]:
merged[((merged['powerBack'].notnull()) & (merged['isMissing'] == True))]

powerBack isMissing  month workDayMF workDayS
id     date_sent                                                        
1013.0 2017-02-06 12:00:00      True      True      2      True    False
       2017-02-17 19:00:00      True      True      2     False    False
1016.0 2017-03-23 04:00:00      True      True      3     False    False
1018.0 2017-01-09 19:00:00     False      True      1     False    False
1020.0 2017-03-23 02:00:00      True      True      3     False    False
1023.0 2017-03-23 05:00:00      True      True      3     False    False
1027.0 2017-03-23 02:00:00      True      True      3     False    False
1033.0 2017-01-07 05:00:00      True      True      1     False    False
       2017-01-11 21:00:00      True      True      1     False    False
       2017-01-13 09:00:00      True      True      1      True    False
       2017-01-16 09:00:00      True      True      1      True    False
       2017-01-17 21:00:00      True      True      1     False    False
       2017-01-18 21:00:00      True      True      1     False    False
       2017-01-19 21:00:00      True      True      1     False    False
       2017-01-24 09:00:00      True      True      1      True    False
       2017-01-26 09:00:00      True      True      1      True    False
       2017-01-29 09:00:00      True      True      1     False     True
       2017-01-30 09:00:00      True      True      1      True    False
       2017-02-01 21:00:00      True      True      2     False    False
1042.0 2017-01-08 00:00:00      True      True      1     False    False
1045.0 2017-03-22 22:00:00      True      True      3     False    False
1058.0 2016-11-29 10:00:00     False      True     11      True    False
1059.0 2017-02-13 13:00:00     False      True      2      True    False
1065.0 2017-03-15 22:00:00      True      True      3     False    False
1069.0 2017-03-23 13:00:00     False      True      3     False    False
1071.0 2017-03-23 02:00:00      True      True      3     False    False
1073.0 2016-11-10 23:00:00     False      True     11     False    False
       2016-12-09 20:00:00     False      True     12     False    False
1085.0 2016-11-29 09:00:00     False      True     11      True    False
1092.0 2017-03-23 01:00:00      True      True      3     False    False
...                              ...       ...    ...       ...      ...
1269.0 2017-03-23 01:00:00      True      True      3     False    False
1273.0 2017-01-01 05:00:00      True      True      1     False    False
1281.0 2017-01-22 04:00:00      True      True      1     False    False
       2017-01-25 16:00:00      True      True      1      True    False
       2017-01-26 16:00:00      True      True      1      True    False
       2017-03-02 16:00:00      True      True      3      True    False
       2017-03-06 16:00:00      True      True      3      True    False
       2017-03-23 04:00:00      True      True      3     False    False
1283.0 2017-01-14 05:00:00      True      True      1     False    False
1288.0 2017-01-10 23:00:00     False      True      1     False    False
       2017-01-19 18:00:00     False      True      1     False    False
1290.0 2017-03-23 05:00:00      True      True      3     False    False
1294.0 2016-12-07 08:00:00     False      True     12     False    False
       2016-12-08 08:00:00     False      True     12     False    False
1297.0 2017-03-23 05:00:00      True      True      3     False    False
1299.0 2016-11-25 10:00:00     False      True     11      True    False
1300.0 2017-02-02 01:00:00      True      True      2     False    False
       2017-02-03 01:00:00      True      True      2     False    False
       2017-02-04 01:00:00      True      True      2     False    False
1301.0 2017-01-16 10:00:00      True      True      1      True    False
       2017-01-22 10:00:00      True      True      1     False     True
       2017-01-24 22:00:00      True      True      1     False    False
       2017-

In [19]:
data[data.id==1033]

,address,body,date_sent,readable_date,service_center,read,type,id,ClusterId,DistrictCity,...,COUNT,PSU_ID,URB_RUR,REGION,DISTRICT,LONG,LAT,dateCollectionStartDays,_merge,dateSentDays
109669,+992935710534,Power Failure detected.ID 1033,2016-12-08 22:11:02,2016-12-08 22:11:04,+992939999999,0,1,1033.0,37.0,Vahdat,...,122.0,37.0,Rural,DRS,Vahdat,69.007561,38.635058,2016-11-19,both,2016-12-08
109670,+992935710534,Power Failure is running ID 1033 | Monitoring...,2016-12-09 08:29:03,2016-12-09 08:29:06,+992939999999,0,1,1033.0,37.0,Vahdat,...,122.0,37.0,Rural,DRS,Vahdat,69.007561,38.635058,2016-11-19,both,2016-12-09
109671,+992935710534,Power Failure detected.ID 1033,2016-12-09 10:01:27,2016-12-09 10:03:06,+992939999999,0,1,1033.0,37.0,Vahdat,...,122.0,37.0,Rural,DRS,Vahdat,69.007561,38.635058,2016-11-19,both,2016-12-09
109672,+992935710534,Power Back detected.ID 1033,2016-12-09 17:01:32,2016-12-09 17:02:54,+992939999999,0,1,1033.0,37.0,Vahdat,...,122.0,37.0,Rural,DRS,Vahdat,69.007561,38.635058,2016-11-19,both,2016-12-09
109673,+992935710534,Power Back detected.ID 1033,2016-12-10 17:53:49,2016-12-10 17:53:53,+992939999999,0,1,1033.0,37.0,Vahdat,...,122.0,37.0,Rural,DRS,Vahdat,69.007561,38.635058,2016-11-19,both,2016-12-10
109674,+992935710534,Power Failure detected.ID 1033,2016-12-10 17:53:54,2016-12-10 17:53:59,+992939999999,0,1,1033.0,37.0,Vahdat,...,122.0,37.0,Rural,DRS,Vahdat,69.007561,38.635058,2016-11-19,both,2016-12-10
109675,+992935710534,Power Failure detected.ID 1033,2016-12-11 19:08:37,2016-12-11 19:08:40,+992939999999,0,1,1033.0,37.0,Vahdat,...,122.0,37.0,Rural,DRS,Vahdat,69.007561,38.635058,2016-11-19,both,2016-12-11
109676,+992935710534,Power Back detected.ID 1033,2016-12-11 19:16:30,2016-12-11 19:16:33,+992939999999,0,1,1033.0,37.0,Vahdat,...,122.0,37.0,Rural,DRS,Vahdat,69.007561,38.635058,2016-11-19,both,2016-12-11
109677,+992935710534,Power Failure detected.ID 1033,2016-12-11 19:16:51,2016-12-11 19:16:54,+992939999999,0,1,1033.0,37.0,Vahdat,...,122.0,37.0,Rural,DRS,Vahdat,69.007561,38.635058,2016-11-19,both,2016-12-11
109678,+992935710534,Power Back detected.ID 1033,2016-12-11 19:33:17,2016-12-11 19:33:20,+992939999999,0,1,1033.0,37.0,Vahdat,...,122.0,37.0,Rural,DRS,Vahdat,69.007561,38.635058,2016-11-19,both,2016-12-11


In [20]:
"""Power status is obviously missing from hours that have been marked as missing
We will *temporarily* fill them in as True (as having power), as that is the assumed default state
This we do for purposes of filling powerBack hours in between body text messages
However, we don't want to lose the info in those hours when there's a message saying "power failure"
In between three days of no messages. That's why we advocated for labeling as not missing 24 hours around
any message. As a result, we won't change the status of those and won't impure them
"""
# if hour is marked missing and there is no power info, we'll mark the hour as True for filling up hours
# that are NOT missing, but border with missing hours. We will then make those missing again

merged.reset_index(inplace=True)

boxList = set(merged['id'])
holder = pd.DataFrame(columns=merged.columns)
for box in boxList:
    this = merged[merged['id'] == box]
    mask = (this['powerBack'].isnull()) & (this['isMissing'] == True)
    this['powerBack'][mask] = True 
    this['powerBack'].fillna(method='ffill', inplace=True)
    this['powerBack'][mask] = np.nan
    this['powerBack'][this['powerBack'].isnull() & (this['isMissing'] == False)] = 1
    this['powerBack'] = this['powerBack'].astype('bool')
    holder = pd.concat([holder, this], ignore_index=True)

merged = holder.copy()

/home/ram22/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ram22/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py:4702: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/ram22/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/ram22/anaconda3/lib/pyt

In [21]:
merged.to_pickle('dataForImputation.pandas')